# 2020 US Election Progress Analysis

https://www.theguardian.com/us-news/ng-interactive/2020/nov/07/us-election-2020-live-results-donald-trump-joe-biden-presidential-votes-pennsylvania-georgia-arizona-nevada

In [1]:
import requests
import pandas as pd
import datetime
import dateutil.parser
import json
import re
import glob
import sys
import locale
locale.setlocale(locale.LC_ALL, '')  # Use '' for auto, or force e.g. to 'en_US.UTF-8'

# gimme 100% width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def format_percent(x):
    return f'{x*100:.2f}'

def flattened(l):
    return [item for sublist in l for item in sublist]

NAME_BY_ACRONYM = {
    'AL': 'Alabama',        'AK': 'Alaska',        'AZ': 'Arizona',    'AR': 'Arkansas',             'CA': 'California',
    'CO': 'Colorado',       'CT': 'Connecticut',   'DE': 'Delaware',   'DC': 'District of Columbia', 'FL': 'Florida',
    'GA': 'Georgia',        'HI': 'Hawaii',        'ID': 'Idaho',      'IL': 'Illinois',             'IN': 'Indiana',
    'IA': 'Iowa',           'KS': 'Kansas',        'KY': 'Kentucky',   'LA': 'Louisiana',            'ME': 'Maine',
    'MD': 'Maryland',       'MA': 'Massachusetts', 'MI': 'Michigan',   'MN': 'Minnesota',            'MS': 'Mississippi',
    'MO': 'Missouri',       'MT': 'Montana',       'NE': 'Nebraska',   'NV': 'Nevada',               'NH': 'New Hampshire',
    'NJ': 'New Jersey',     'NM': 'New Mexico',    'NY': 'New York',   'NC': 'North Carolina',       'ND': 'North Dakota',
    'OH': 'Ohio',           'OK': 'Oklahoma',      'OR': 'Oregon',     'PA': 'Pennsylvania',         'RI': 'Rhode Island',
    'SC': 'South Carolina', 'SD': 'South Dakota',  'TN': 'Tennessee',  'TX': 'Texas',    'UT':       'Utah',
    'VT': 'Vermont',        'VA': 'Virginia',      'WA': 'Washington', 'WV': 'West Virginia',        'WI': 'Wisconsin',
    'WY': 'Wyoming',        'PR': 'Puerto Rico',
}

def load(file_name=None):
    if file_name:
        with open(file_name) as f:
            return json.load(f)
    
    # Load dataset
    try:
        # From the Guardian on the Web...
        last_update = requests.get('https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/last_updated.json').json()
        last_time = last_update['time']
        url = f'https://interactive.guim.co.uk/2020/11/us-general-election-data/prod/data-out/{last_time}/president_details.json'
        print(last_time)
        data = requests.get(url).json()
        fname = 'data-cache--' + re.compile('[^0-9a-zA-Z-]').sub('.', last_time) + '.json'
        with open(fname, 'w') as f: json.dump(data, f, indent='  ', sort_keys=True)
    except Exception as e:
        # ... or cached
        print(e, file=sys.stderr)
        names = sorted(glob.glob('data-cache--*.json'))
        print('Found cache:', names)
        name = names[-1]
        print('Loading:', name)
        with open(name) as f: data = json.load(f)
    return data

def compute(data):
    electors_by_candidate = {}
    electors_by_candidate_may = {}
    electors_by_state = {}

    new_data = {}
    new_data_short = {}

    n = 0

    # Compute electors by state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        electors = max( c['electWon'] for c in state_data['candidates'])
        # Some return zero, fix that
        electors_by_state[acronym] = {'AK': 6, 'GA': 16}.get(acronym, electors)

    # Analyze each state
    for acronym, state_data in data.items():
        if acronym == 'US': continue
        n += 1
        extra = {}
        extra_short = {}
        electors = electors_by_state[acronym]

        # Analyze each field
        for k, v in state_data.items():
            if k == 'declarationTime' and v:
                # make it shorter
                dt = dateutil.parser.parse(v)
                state_data[k] = datetime.datetime.strftime(dt, '%b %d %H:%M')
            if k == 'candidates':
                candidate = {c['name'].split()[-1]:c for c in v}
                if n and n < 4:
                    pass #print();print(candidate)
                # compute votes
                nb_counted = sum([c['votes'] for c in v])
                max_votes = int(state_data['totalVotes'] / state_data['reporting'] * 100.)
                rem_votes = max_votes - nb_counted
                # general fields
                #extra['electors'] = electors
                extra['% counted']   = extra_short['% counted']   = state_data['reporting']
                extra['grand total'] = extra_short['grand total'] = max_votes
                extra['uncounted']   = extra_short['uncounted']   = rem_votes
                # per-candidate votes
                extra['- SCORE -'] = ''
                for name, cdata in candidate.items():
                    val = cdata['votes'] / nb_counted
                    extra[name]       = extra_short[name]       = cdata['votes']
                for name, cdata in candidate.items():
                    val = cdata['votes'] / nb_counted
                    extra[name + '%']       = format_percent(val)
                    extra_short[name + '%'] = val
                # winner
                winner = sorted([(cdata['votes'], name) for name, cdata in candidate.items()])[-1][1]
                extra['winner'] = winner
                # compute "maybe" winner: who could win, if getting all remaining votes
                extra['- MAYBE -'] = ''
                cand2 = {c['votes']:c['name'].split()[-1] for c in v}
                possibilities = [
                    [[votes2 + (rem_votes if name2!=winner else 0), name2] for votes2, name2 in cand2.items()]
                    for votes1, name1 in cand2.items()]
                winner_may_votes, winner_may = sorted(flattened(possibilities))[-1]
                for name, cdata in candidate.items():
                    val = (cdata['votes']+rem_votes) / (nb_counted+rem_votes)
                    extra[name + '%may'] = format_percent(val) if name != winner else f'({format_percent(val)})'
                extra['may'] = winner_may if winner_may != winner else ''
                #extra['may votes'] = winner_may_votes
                #extra['may %'] = format_percent(winner_may_votes / max_votes)

        # accumulate won electors
        electors_by_candidate.setdefault(winner, 0)
        electors_by_candidate[winner] += electors
        electors_by_candidate_may.setdefault(winner_may, 0)
        electors_by_candidate_may[winner_may] += electors

        # format ints
        for k, v in extra.items():
            if v.__class__ is int:
                extra[k] = f'{v:n}'
        # keep new data and format ints
        new_data[acronym] = extra
        new_data_short[acronym] = extra_short

    electors_by_candidate = [{'Candidate': cand, 'Electors': electors} for cand, electors in electors_by_candidate.items()]
    electors_by_candidate_may = [{'Candidate maybe': cand, 'Electors': electors} for cand, electors in electors_by_candidate_may.items()]
    electors_by_state = {acronym:{'Name': NAME_BY_ACRONYM[acronym], 'Electors': nb} for acronym, nb in electors_by_state.items()}
    
    return new_data, new_data_short, electors_by_candidate, electors_by_candidate_may, electors_by_state

def dic_dict_sub(d1, d2):
    def fmt(v):
        if v.__class__ == int:
            return f'{v:n}'
        elif v.__class__ == float:
            return f'{v*100:.2f}'
        else:
            return v
        
    return { k1:{k2:fmt(v2-d2[k1][k2]) for k2, v2 in v1.items()} for k1, v1 in d1.items()}

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)

# 1. Results as of Sat 7 Nov

In [2]:
data = load('data-cache--2020-11-07T17.14.12.533.00.00.json')

new_data, new_data_short_0, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,50,99,99,90,66,95,99,90,99,99,99,96,99,99,92,99,99,99,95,99,86,99,99,99,99,99,99,99,99,99,99,77,99,92,78,96,99,98,99,99,99,94,99,98,93,99,92,85,99,99,97
grand total,381'940,2'313'155,1'220'137,3'659'095,21'489'998,3'340'132,1'788'948,319'751,507'418,11'155'090,5'033'023,597'764,1'700'530,875'985,5'950'636,3'054'370,1'346'982,2'155'455,2'260'329,3'564'678,2'859'958,821'532,5'575'096,3'301'777,3'038'125,1'144'988,606'747,5'513'657,362'294,945'036,809'828,4'486'316,926'419,1'408'439,8'503'584,6'001'302,1'574'370,2'335'332,6'815'155,510'304,2'538'256,449'571,3'076'244,11'470'733,1'397'625,4'419'261,397'220,4'608'967,3'322'701,793'051,283'373
uncounted,194'148,23'131,33'386,365'909,7'446'895,200'994,25'096,34'519,7'209,141'744,50'330,29'835,27'167,18'574,518'060,30'543,13'469,31'625,127'615,53'781,418'071,17'831,79'636,64'282,42'494,20'177,6'067,74'398,3'622,9'450,8'098,1'049'835,17'052,128'007,1'911'701,258'084,27'525,62'723,68'151,7'582,34'148,26'974,57'551,262'556,117'298,44'192,38'154,715'333,43'686,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,118'602,1'430'589,755'815,1'612'585,4'741'191,1'335'253,714'981,15'075,199'829,5'658'847,2'455'305,196'602,896'100,554'019,2'316'566,1'725'723,752'903,1'325'742,1'255'481,1'148'777,857'843,359'502,2'644'525,1'484'671,1'711'848,677'218,341'767,2'733'645,234'845,548'462,365'373,1'385'529,400'095,619'944,2'849'085,3'074'418,1'018'870,928'706,3'314'164,197'421,1'384'852,261'035,1'849'211,5'865'248,759'638,1'946'870,112'688,1'514'563,1'610'030,539'610,193'454
Biden,63'992,834'533,417'897,1'631'195,9'163'725,1'753'416,1'029'132,268'625,295'403,5'284'453,2'465'500,365'802,757'707,286'991,3'056'219,1'239'401,551'144,771'884,855'597,2'316'338,1'557'743,430'023,2'790'648,1'717'830,1'242'851,440'284,243'719,2'658'274,114'480,367'319,423'291,2'029'493,496'826,647'474,3'694'996,2'603'731,503'289,1'304'536,3'355'387,300'325,1'091'348,150'467,1'139'666,5'217'656,489'469,2'364'415,242'805,2'303'430,1'630'570,232'502,73'445
Jorgensen,5'198,24'902,13'039,49'406,138'187,50'469,19'739,1'532,4'977,70'046,61'888,5'525,19'556,16'401,59'791,58'703,29'466,26'204,21'636,45'782,26'301,14'176,60'287,34'994,40'932,7'309,15'194,47'340,9'347,19'805,13'066,21'459,12'446,13'014,47'802,65'069,24'686,39'367,77'453,4'976,27'908,11'095,29'816,125'273,31'220,63'784,3'573,75'641,38'415,10'462,5'765
Trump%,63.16,62.47,63.69,48.97,33.76,42.54,40.54,5.29,39.95,51.38,49.28,34.62,53.55,64.62,42.64,57.07,56.46,62.42,58.87,32.72,35.13,44.73,48.12,45.86,57.14,60.21,56.90,50.26,65.48,58.62,45.57,40.32,44.00,48.42,43.22,53.53,65.87,40.87,49.12,39.27,55.30,61.77,61.26,52.33,59.33,44.50,31.38,38.90,49.10,68.95,70.95
Biden%,34.08,36.44,35.21,49.53,65.25,55.86,58.35,94.18,59.06,47.98,49.48,64.41,45.28,33.47,56.26,40.99,41.33,36.34,40.12,65.98,63.79,53.51,50.78,53.06,41.49,39.14,40.57,48.87,31.92,39.26,52.80,59.06,54.63,50.57,56.05,45.34,32.54,57.40,49.73,59.74,43.58,35.61,37.75,46.55,38.23,54.04,67.62,59.16,49.73,29.71,26.94
Jorgensen%,2.77,1.09,1.10,1.50,0.98,1.61,1.12,0.54,0.99,0.64,1.24,0.97,1.17,1.91,1.10,1.94,2.21,1.23,1.01,1.30,1.08,1.76,1.10,1.08,1.37,0.65,2.53,0.87,2.61,2.12,1.63,0.62,1.37,1.02,0.73,1.13,1.60,1.73,1.15,0.99,1.11,2.63,0.99,1.12,2.44,1.46,1.00,1.94,1.17,1.34,2.11


In [3]:
pd.DataFrame(electors_by_state)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Name,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,District of Columbia,Delaware,Florida,Georgia,Hawaii,Iowa,Idaho,Illinois,Indiana,Kansas,Kentucky,Louisiana,Massachusetts,Maryland,Maine,Michigan,Minnesota,Missouri,Mississippi,Montana,North Carolina,North Dakota,Nebraska,New Hampshire,New Jersey,New Mexico,Nevada,New York,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming
Electors,6,9,6,11,55,9,7,3,3,29,16,4,6,4,20,11,6,8,8,11,10,3,16,10,10,6,3,0,3,4,4,14,5,6,29,18,7,7,20,4,9,3,11,38,6,13,3,12,10,5,3


## Electors

In [4]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,219
1,Biden,305


## Potential twist 

In [5]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Biden,150
1,Trump,374


# 2. Results as of Thu 12 Nov

In [6]:
#data = load('data-cache--2020-11-09T23.58.18.360.00.00.json')
#data = load('data-cache--2020-11-11T07.57.21.383.00.00.json')
#data = load('data-cache--2020-11-12T12.27.02.803.00.00.json')
data = load('data-cache--2020-11-13T09.09.56.007.00.00.json')
new_data, new_data_short_1, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,92,99,99,99,93,95,99,99,99,99,99,96,99,99,92,99,99,99,95,99,97,99,99,99,99,99,99,99,99,99,99,91,99,95,80,96,99,98,99,99,99,94,99,99,98,99,99,99,99,99,97
grand total,351'613,2'313'155,1'220'137,3'411'325,17'571'921,3'340'132,1'839'219,309'639,507'418,11'155'090,5'042'486,597'764,1'702'241,875'985,5'950'539,3'054'370,1'346'982,2'156'561,2'260'329,3'564'678,2'849'018,821'532,5'575'096,3'303'076,3'038'125,1'144'988,606'747,5'540'270,363'525,947'480,809'828,4'508'552,926'419,1'407'749,8'374'958,6'001'302,1'574'370,2'335'332,6'894'997,510'304,2'538'256,449'571,3'076'244,11'356'541,1'477'968,4'441'047,369'134,4'044'885,3'322'701,793'051,283'373
uncounted,32'494,23'131,33'386,34'113,1'404'186,200'994,25'919,5'910,7'209,141'744,50'424,29'835,27'162,18'574,517'963,30'543,13'469,31'641,127'615,53'781,105'139,17'831,79'636,64'146,42'494,20'177,6'067,74'953,3'635,9'474,8'098,428'760,17'052,86'763,1'716'602,258'084,27'525,62'723,68'949,7'582,34'148,26'974,57'551,146'715,52'623,44'410,10'068,65'395,43'686,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,173'233,1'430'589,755'815,1'657'250,5'536'973,1'335'253,713'778,16'306,199'829,5'658'847,2'457'920,196'602,897'140,554'019,2'316'566,1'725'723,752'903,1'326'418,1'255'481,1'148'777,926'771,359'502,2'644'525,1'485'254,1'711'848,677'218,341'767,2'744'299,235'595,550'202,365'373,1'698'349,400'095,635'089,2'871'533,3'074'418,1'018'870,928'706,3'343'692,197'421,1'384'852,261'035,1'849'211,5'865'913,840'163,1'952'834,112'688,1'556'739,1'610'030,539'610,193'454
Biden,138'070,834'533,417'897,1'668'684,10'460'073,1'753'416,1'079'223,285'728,295'403,5'284'453,2'472'083,365'802,758'352,286'991,3'056'219,1'239'401,551'144,772'285,855'597,2'316'338,1'786'733,430'023,2'790'648,1'718'683,1'242'851,440'284,243'719,2'672'900,114'902,367'919,423'291,2'353'302,496'826,671'955,3'738'403,2'603'731,503'289,1'304'536,3'403'925,300'325,1'091'348,150'467,1'139'666,5'218'631,547'976,2'379'830,242'805,2'344'336,1'630'570,232'502,73'445
Jorgensen,7'816,24'902,13'039,51'278,170'689,50'469,20'299,1'695,4'977,70'046,62'059,5'525,19'587,16'401,59'791,58'703,29'466,26'217,21'636,45'782,30'375,14'176,60'287,34'993,40'932,7'309,15'194,48'118,9'393,19'885,13'066,28'141,12'446,13'942,48'420,65'069,24'686,39'367,78'431,4'976,27'908,11'095,29'816,125'282,37'206,63'973,3'573,78'415,38'415,10'462,5'765
Trump%,54.28,62.47,63.69,49.07,34.25,42.54,39.36,5.37,39.95,51.38,49.24,34.62,53.56,64.62,42.64,57.07,56.46,62.42,58.87,32.72,33.78,44.73,48.12,45.86,57.14,60.21,56.90,50.21,65.46,58.66,45.57,41.63,44.00,48.08,43.13,53.53,65.87,40.87,48.98,39.27,55.30,61.77,61.26,52.33,58.94,44.42,31.38,39.12,49.10,68.95,70.95
Biden%,43.27,36.44,35.21,49.41,64.70,55.86,59.52,94.07,59.06,47.98,49.52,64.41,45.27,33.47,56.26,40.99,41.33,36.34,40.12,65.98,65.12,53.51,50.78,53.06,41.49,39.14,40.57,48.91,31.93,39.22,52.80,57.68,54.63,50.87,56.15,45.34,32.54,57.40,49.87,59.74,43.58,35.61,37.75,46.55,38.45,54.13,67.62,58.91,49.73,29.71,26.94
Jorgensen%,2.45,1.09,1.10,1.52,1.06,1.61,1.12,0.56,0.99,0.64,1.24,0.97,1.17,1.91,1.10,1.94,2.21,1.23,1.01,1.30,1.11,1.76,1.10,1.08,1.37,0.65,2.53,0.88,2.61,2.12,1.63,0.69,1.37,1.06,0.73,1.13,1.60,1.73,1.15,0.99,1.11,2.63,0.99,1.12,2.61,1.46,1.00,1.97,1.17,1.34,2.11


In [16]:
pd.DataFrame(electors_by_state)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Name,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,District of Columbia,Delaware,Florida,Georgia,Hawaii,Iowa,Idaho,Illinois,Indiana,Kansas,Kentucky,Louisiana,Massachusetts,Maryland,Maine,Michigan,Minnesota,Missouri,Mississippi,Montana,North Carolina,North Dakota,Nebraska,New Hampshire,New Jersey,New Mexico,Nevada,New York,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming
Electors,6,9,6,11,55,9,7,3,3,29,16,4,6,4,20,11,6,8,8,11,10,3,16,10,10,0,3,15,3,4,4,14,5,6,0,18,7,7,20,4,9,3,11,38,6,13,3,12,10,5,3


## Electors

In [7]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,219
1,Biden,305


## Potential twist 

In [8]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Trump,311
1,Biden,213


# 3. Results as of now

In [9]:
data = load()
new_data, new_data_short_2, electors_by_candidate, electors_by_candidate_may, electors_by_state = compute(data)
pd.DataFrame(new_data)

2020-11-15T14:09:32.663+00:00


,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,96,99,99,99,96,99,99,99,99,99,99,96,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,94,99,99,81,96,99,99,99,99,99,94,99,99,99,99,99,99,99,99,97
grand total,353'171,2'326'300,1'222'622,3'419'389,17'571'482,3'288'950,1'839'219,333'938,507'418,11'177'313,5'042'361,597'764,1'702'735,875'985,5'989'964,3'061'127,1'346'982,2'156'561,2'169'759,3'564'678,2'908'461,821'532,5'584'322,3'298'764,3'038'125,1'270'345,609'880,5'564'956,363'525,955'376,809'828,4'488'318,930'823,1'419'571,8'368'865,6'001'302,1'574'370,2'360'359,6'918'882,518'789,2'538'256,449'571,3'076'244,11'356'541,1'478'647,4'474'953,369'134,4'074'355,3'322'950,793'051,283'373
uncounted,18'730,23'263,33'445,34'193,885'274,68'400,25'919,6'524,7'209,142'069,50'423,29'835,27'173,18'574,106'842,30'611,13'469,31'641,36'304,53'781,49'465,17'831,79'733,64'058,42'494,21'949,6'098,75'386,3'635,9'553,8'098,293'237,17'148,31'412,1'632'401,258'084,27'525,40'188,69'188,7'720,34'148,26'974,57'551,146'715,38'143,44'749,10'068,65'987,43'636,10'477,10'709
- SCORE -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Trump,181'645,1'434'315,757'315,1'661'677,5'748'735,1'364'202,713'778,18'172,199'829,5'668'457,2'457'880,196'602,897'356,554'019,2'404'069,1'729'472,752'903,1'326'418,1'255'776,1'148'777,951'838,359'502,2'649'063,1'483'551,1'711'848,734'466,343'643,2'757'112,235'595,553'793,365'373,1'754'039,401'361,669'890,2'888'417,3'074'418,1'018'870,949'646,3'352'569,199'830,1'384'852,261'035,1'849'211,5'865'913,848'863,1'961'131,112'688,1'572'345,1'610'139,539'610,193'454
Biden,144'498,843'724,418'811,1'672'054,10'758'123,1'803'921,1'079'223,307'265,295'403,5'296'482,2'472'002,365'802,758'597,286'991,3'413'787,1'242'163,551'144,772'285,856'034,2'316'338,1'875'270,430'023,2'795'184,1'716'207,1'242'851,506'104,244'833,2'683'787,114'902,371'921,423'291,2'411'634,499'782,703'486,3'798'896,2'603'731,503'289,1'329'567,3'418'435,306'192,1'091'348,150'467,1'139'666,5'218'631,554'028,2'404'499,242'805,2'356'584,1'630'683,232'502,73'445
Jorgensen,8'298,24'998,13'051,51'465,179'350,52'427,20'299,1'977,4'977,70'305,62'056,5'525,19'609,16'401,65'266,58'881,29'466,26'217,21'645,45'782,31'888,14'176,60'342,34'948,40'932,7'826,15'306,48'671,9'393,20'109,13'066,29'408,12'532,14'783,49'151,65'069,24'686,40'958,78'690,5'047,27'908,11'095,29'816,125'282,37'613,64'574,3'573,79'439,38'492,10'462,5'765
Trump%,54.31,62.28,63.68,49.09,34.45,42.36,39.36,5.55,39.95,51.37,49.24,34.62,53.56,64.62,40.86,57.07,56.46,62.42,58.86,32.72,33.29,44.73,48.12,45.86,57.14,58.83,56.92,50.22,65.46,58.55,45.57,41.81,43.93,48.26,42.88,53.53,65.87,40.93,48.94,39.10,55.30,61.77,61.26,52.33,58.93,44.27,31.38,39.23,49.10,68.95,70.95
Biden%,43.21,36.64,35.22,49.39,64.47,56.01,59.52,93.85,59.06,48.00,49.52,64.41,45.27,33.47,58.03,40.99,41.33,36.34,40.12,65.98,65.59,53.51,50.78,53.06,41.49,40.54,40.55,48.89,31.93,39.32,52.80,57.49,54.70,50.68,56.39,45.34,32.54,57.30,49.91,59.91,43.58,35.61,37.75,46.55,38.46,54.28,67.62,58.79,49.73,29.71,26.94
Jorgensen%,2.48,1.09,1.10,1.52,1.07,1.63,1.12,0.60,0.99,0.64,1.24,0.97,1.17,1.91,1.11,1.94,2.21,1.23,1.01,1.30,1.12,1.76,1.10,1.08,1.37,0.63,2.54,0.89,2.61,2.13,1.63,0.70,1.37,1.06,0.73,1.13,1.60,1.77,1.15,0.99,1.11,2.63,0.99,1.12,2.61,1.46,1.00,1.98,1.17,1.34,2.11


In [15]:
pd.DataFrame(electors_by_state)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Name,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,District of Columbia,Delaware,Florida,Georgia,Hawaii,Iowa,Idaho,Illinois,Indiana,Kansas,Kentucky,Louisiana,Massachusetts,Maryland,Maine,Michigan,Minnesota,Missouri,Mississippi,Montana,North Carolina,North Dakota,Nebraska,New Hampshire,New Jersey,New Mexico,Nevada,New York,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming
Electors,6,9,6,11,55,9,7,3,3,29,16,4,6,4,20,11,6,8,8,11,10,3,16,10,10,0,3,15,3,4,4,14,5,6,0,18,7,7,20,4,9,3,11,38,6,13,3,12,10,5,3


## Progression 

In [10]:
pd.DataFrame(dic_dict_sub(new_data_short_2, new_data_short_1))

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,4,0,0,0,3,4,0,0,0,0,0,0,0,0,7,0,0,0,4,0,2,0,0,0,0,0,0,0,0,0,0,3,0,4,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
grand total,1'558,13'145,2'485,8'064,-439,-51'182,0,24'299,0,22'223,-125,0,494,0,39'425,6'757,0,0,-90'570,0,59'443,0,9'226,-4'312,0,125'357,3'133,24'686,0,7'896,0,-20'234,4'404,11'822,-6'093,0,0,25'027,23'885,8'485,0,0,0,0,679,33'906,0,29'470,249,0,0
uncounted,-13'764,132,59,80,-518'912,-132'594,0,614,0,325,-1,0,11,0,-411'121,68,0,0,-91'311,0,-55'674,0,97,-88,0,1'772,31,433,0,79,0,-135'523,96,-55'351,-84'201,0,0,-22'535,239,138,0,0,0,0,-14'480,339,0,592,-50,0,0
Trump,8'412,3'726,1'500,4'427,211'762,28'949,0,1'866,0,9'610,-40,0,216,0,87'503,3'749,0,0,295,0,25'067,0,4'538,-1'703,0,57'248,1'876,12'813,0,3'591,0,55'690,1'266,34'801,16'884,0,0,20'940,8'877,2'409,0,0,0,0,8'700,8'297,0,15'606,109,0,0
Biden,6'428,9'191,914,3'370,298'050,50'505,0,21'537,0,12'029,-81,0,245,0,357'568,2'762,0,0,437,0,88'537,0,4'536,-2'476,0,65'820,1'114,10'887,0,4'002,0,58'332,2'956,31'531,60'493,0,0,25'031,14'510,5'867,0,0,0,0,6'052,24'669,0,12'248,113,0,0
Jorgensen,482,96,12,187,8'661,1'958,0,282,0,259,-3,0,22,0,5'475,178,0,0,9,0,1'513,0,55,-45,0,517,112,553,0,224,0,1'267,86,841,731,0,0,1'591,259,71,0,0,0,0,407,601,0,1'024,77,0,0
Trump%,0.03,-0.19,-0.00,0.02,0.20,-0.18,0.00,0.18,0.00,-0.01,0.00,0.00,-0.00,0.00,-1.78,-0.00,0.00,0.00,-0.01,0.00,-0.48,0.00,0.00,0.01,0.00,-1.37,0.02,0.01,0.00,-0.11,0.00,0.18,-0.07,0.18,-0.25,0.00,0.00,0.06,-0.04,-0.17,0.00,0.00,0.00,0.00,-0.02,-0.15,0.00,0.11,-0.00,0.00,0.00
Biden%,-0.06,0.19,0.01,-0.02,-0.22,0.16,0.00,-0.23,0.00,0.01,-0.00,0.00,0.00,0.00,1.77,0.00,0.00,0.00,0.01,0.00,0.47,0.00,-0.00,-0.01,0.00,1.40,-0.02,-0.02,0.00,0.10,0.00,-0.19,0.07,-0.19,0.25,0.00,0.00,-0.10,0.04,0.17,0.00,0.00,0.00,0.00,0.02,0.15,0.00,-0.12,-0.00,0.00,0.00
Jorgensen%,0.03,-0.00,-0.00,0.00,0.02,0.02,0.00,0.05,0.00,0.00,-0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,-0.00,0.00,0.00,-0.02,0.01,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.03,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00


## Progression since Sat 7 Nov

In [11]:
pd.DataFrame(dic_dict_sub(new_data_short_2, new_data_short_0))

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
% counted,46,0,0,9,30,4,0,9,0,0,0,0,0,0,7,0,0,0,4,0,13,0,0,0,0,0,0,0,0,0,0,17,0,7,3,0,0,1,0,0,0,0,0,1,6,0,7,14,0,0,0
grand total,-28'769,13'145,2'485,-239'706,-3'918'516,-51'182,50'271,14'187,0,22'223,9'338,0,2'205,0,39'328,6'757,0,1'106,-90'570,0,48'503,0,9'226,-3'013,0,125'357,3'133,51'299,1'231,10'340,0,2'002,4'404,11'132,-134'719,0,0,25'027,103'727,8'485,0,0,0,-114'192,81'022,55'692,-28'086,-534'612,249,0,0
uncounted,-175'418,132,59,-331'716,-6'561'621,-132'594,823,-27'995,0,325,93,0,6,0,-411'218,68,0,16,-91'311,0,-368'606,0,97,-224,0,1'772,31,988,13,103,0,-756'598,96,-96'595,-279'300,0,0,-22'535,1'037,138,0,0,0,-115'841,-79'155,557,-28'086,-649'346,-50,0,0
Trump,63'043,3'726,1'500,49'092,1'007'544,28'949,-1'203,3'097,0,9'610,2'575,0,1'256,0,87'503,3'749,0,676,295,0,93'995,0,4'538,-1'120,0,57'248,1'876,23'467,750,5'331,0,368'510,1'266,49'946,39'332,0,0,20'940,38'405,2'409,0,0,0,665,89'225,14'261,0,57'782,109,0,0
Biden,80'506,9'191,914,40'859,1'594'398,50'505,50'091,38'640,0,12'029,6'502,0,890,0,357'568,2'762,0,401,437,0,317'527,0,4'536,-1'623,0,65'820,1'114,25'513,422,4'602,0,382'141,2'956,56'012,103'900,0,0,25'031,63'048,5'867,0,0,0,975,64'559,40'084,0,53'154,113,0,0
Jorgensen,3'100,96,12,2'059,41'163,1'958,560,445,0,259,168,0,53,0,5'475,178,0,13,9,0,5'587,0,55,-46,0,517,112,1'331,46,304,0,7'949,86,1'769,1'349,0,0,1'591,1'237,71,0,0,0,9,6'393,790,0,3'798,77,0,0
Trump%,-8.84,-0.19,-0.00,0.12,0.69,-0.18,-1.17,0.26,0.00,-0.01,-0.04,0.00,0.00,0.00,-1.78,-0.00,0.00,-0.00,-0.01,0.00,-1.84,0.00,0.00,0.00,0.00,-1.37,0.02,-0.03,-0.01,-0.07,0.00,1.49,-0.07,-0.16,-0.34,0.00,0.00,0.06,-0.18,-0.17,0.00,0.00,0.00,-0.00,-0.40,-0.23,0.00,0.33,-0.00,0.00,0.00
Biden%,9.13,0.19,0.01,-0.14,-0.78,0.16,1.17,-0.33,0.00,0.01,0.04,0.00,-0.01,0.00,1.77,0.00,0.00,0.00,0.01,0.00,1.80,0.00,-0.00,-0.00,0.00,1.40,-0.02,0.02,0.01,0.06,0.00,-1.57,0.07,0.11,0.34,0.00,0.00,-0.10,0.17,0.17,0.00,0.00,0.00,0.00,0.23,0.23,0.00,-0.37,-0.00,0.00,0.00
Jorgensen%,-0.29,-0.00,-0.00,0.02,0.09,0.02,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,-0.00,0.00,0.00,0.04,0.00,-0.00,-0.00,0.00,-0.02,0.01,0.02,0.00,0.01,0.00,0.08,0.00,0.05,0.00,0.00,0.00,0.03,0.00,-0.00,0.00,0.00,0.00,-0.00,0.17,-0.00,0.00,0.04,0.00,0.00,0.00


## Electors

In [12]:
pd.DataFrame(electors_by_candidate)

,Candidate,Electors
0,Trump,228
1,Biden,276


## Potential twist

In [13]:
pd.DataFrame(electors_by_candidate_may)

,Candidate maybe,Electors
0,Trump,270
1,Biden,234


# 4. Electors per state

In [14]:
pd.DataFrame(electors_by_state).transpose()

,Name,Electors
AK,Alaska,6
AL,Alabama,9
AR,Arkansas,6
AZ,Arizona,11
CA,California,55
CO,Colorado,9
CT,Connecticut,7
DC,District of Columbia,3
DE,Delaware,3
FL,Florida,29
